In [1]:
import numpy as np
import glob
import os

import endocytosis.contrib.gohlke.tifffile as tifffile
import endocytosis.io.path as path
import endocytosis.io.image.roi.roi_read as roi_read

In [2]:
folder = r"D:\endocytosis_data_subset\2016-12-14 iso\2016-12-14 Sp57 proto 003"
filepaths = glob.glob(folder + os.path.sep + '*.tif')

In [3]:
filepaths

['D:\\endocytosis_data_subset\\2016-12-14 iso\\2016-12-14 Sp57 proto 003\\2016-12-14 Sp57 proto 003.tif']

In [4]:
im = tifffile.TiffFile(filepaths[0])

In [5]:
pathfinder = path.PathFinder(regexp='RoiSetAuto.*', extension='zip')

In [6]:
zip_filenames = [p[0] for p in pathfinder.load(folder)]

In [7]:
folder + os.path.sep + zip_filenames[0]

'D:\\endocytosis_data_subset\\2016-12-14 iso\\2016-12-14 Sp57 proto 003\\RoiSetAuto-100.zip'

In [8]:
with roi_read.IJZipReader(regexp=r'.*[0-9](#x)?\.roi$', sep=None) as roi_reader:
    roi_reader.read(folder + os.path.sep + zip_filenames[0])
    for k, v in roi_reader.items():
        print(k, v.centroid)

0030-0030-0964#x {'px': array([729.5, 964.5])}
0031-0031-0965 {'px': array([728.5, 965.5])}
0032-0032-0965 {'px': array([730.5, 965.5])}
0033-0033-0967 {'px': array([729.5, 967.5])}
0034-0034-0967 {'px': array([728.5, 967.5])}
0035-0035-0968 {'px': array([726.5, 968.5])}
0036-0036-0970#x {'px': array([726.5, 970.5])}
0037-0037-0970 {'px': array([726.5, 970.5])}
0038-0038-0969 {'px': array([727.5, 969.5])}


In [9]:
bytestreams = roi_reader.bytestreams

In [25]:
from endocytosis.io.image.roi import (HEADER_SIZE, HEADER2_SIZE,
                                      HEADER_DTYPE, HEADER2_DTYPE,
                                      OPTIONS, SUBTYPE, ROI_TYPE)

In [11]:
length = len(bytestreams)
hdr = np.recarray(length, HEADER_DTYPE, buf=b''.join([
                  b[:HEADER_SIZE] for b in bytestreams]))

In [12]:
hdr['hdr2_offset']

array([64, 64, 64, 64, 64, 64, 64, 64, 64])

In [13]:
hdr2 = np.recarray(length, HEADER2_DTYPE, buf=b''.join([
                   b[off:off+HEADER2_SIZE] for off, b in zip(
                       hdr['hdr2_offset'], bytestreams)]))

In [14]:
hdr2['name_offset'].tobytes()

b'\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80'

In [15]:
from struct import unpack
offsets, lengths = hdr2['name_offset'], hdr2['name_length']
names = ["".join(map(chr, unpack('>'+'h'*le, bs[off:off+le*2]))) for
                 bs, off, le in zip(bytestreams, offsets, lengths)]

In [16]:
names

['0030-0030-0964#x',
 '0031-0031-0965',
 '0032-0032-0965',
 '0033-0033-0967',
 '0034-0034-0967',
 '0035-0035-0968',
 '0036-0036-0970#x',
 '0037-0037-0970',
 '0038-0038-0969']

In [17]:
hdr

rec.array([(b'Iout', 227, 2, 962, 727, 967, 732, 0, 0., 0., 0., 0., 0, 0, (0, 0, 0, 0), (0, 0, 0, 0), 0, 0, 0, 0, 0, 0, 64),
           (b'Iout', 227, 2, 963, 726, 968, 731, 0, 0., 0., 0., 0., 0, 0, (0, 0, 0, 0), (0, 0, 0, 0), 0, 0, 0, 0, 0, 0, 64),
           (b'Iout', 227, 2, 963, 728, 968, 733, 0, 0., 0., 0., 0., 0, 0, (0, 0, 0, 0), (0, 0, 0, 0), 0, 0, 0, 0, 0, 0, 64),
           (b'Iout', 227, 2, 965, 727, 970, 732, 0, 0., 0., 0., 0., 0, 0, (0, 0, 0, 0), (0, 0, 0, 0), 0, 0, 0, 0, 0, 0, 64),
           (b'Iout', 227, 2, 965, 726, 970, 731, 0, 0., 0., 0., 0., 0, 0, (0, 0, 0, 0), (0, 0, 0, 0), 0, 0, 0, 0, 0, 0, 64),
           (b'Iout', 227, 2, 966, 724, 971, 729, 0, 0., 0., 0., 0., 0, 0, (0, 0, 0, 0), (0, 0, 0, 0), 0, 0, 0, 0, 0, 0, 64),
           (b'Iout', 227, 2, 968, 724, 973, 729, 0, 0., 0., 0., 0., 0, 0, (0, 0, 0, 0), (0, 0, 0, 0), 0, 0, 0, 0, 0, 0, 64),
           (b'Iout', 227, 2, 968, 724, 973, 729, 0, 0., 0., 0., 0., 0, 0, (0, 0, 0, 0), (0, 0, 0, 0), 0, 0, 0, 0, 0, 0, 64),


In [18]:
from struct import pack
ord_names = [list(map(ord, n)) for n in names]

In [48]:
bnames = [pack('>' + 'h'*len(n), *n) for n in ord_names]

In [49]:
bnames[0] in bytestreams[0]

True

In [52]:
bytestreams[0][128:]

b'\x000\x000\x003\x000\x00-\x000\x000\x003\x000\x00-\x000\x009\x006\x004\x00#\x00x\x00T\x00M\x00_\x00Q\x00u\x00a\x00l\x00i\x00t\x00y\x00:\x00 \x005\x002\x009\x00.\x005\x004\x009\x006\x002\x001\x005\x008\x002\x000\x003\x001\x002\x00\n\x00T\x00M\x00_\x00R\x00a\x00d\x00i\x00u\x00s\x00:\x00 \x000\x00.\x003\x00\n\x00T\x00M\x00_\x00X\x00:\x00 \x009\x004\x00.\x008\x003\x004\x001\x000\x009\x005\x004\x000\x005\x002\x002\x000\x002\x00\n\x00T\x00M\x00_\x00Y\x00:\x00 \x001\x002\x005\x00.\x003\x009\x004\x004\x003\x008\x000\x008\x004\x009\x000\x004\x005\x001\x00\n'

In [51]:
bnames[0]

b'\x000\x000\x003\x000\x00-\x000\x000\x003\x000\x00-\x000\x009\x006\x004\x00#\x00x'

In [53]:
len(bnames[0])

32

In [54]:
bytestreams[0][128+32:]

b'\x00T\x00M\x00_\x00Q\x00u\x00a\x00l\x00i\x00t\x00y\x00:\x00 \x005\x002\x009\x00.\x005\x004\x009\x006\x002\x001\x005\x008\x002\x000\x003\x001\x002\x00\n\x00T\x00M\x00_\x00R\x00a\x00d\x00i\x00u\x00s\x00:\x00 \x000\x00.\x003\x00\n\x00T\x00M\x00_\x00X\x00:\x00 \x009\x004\x00.\x008\x003\x004\x001\x000\x009\x005\x004\x000\x005\x002\x002\x000\x002\x00\n\x00T\x00M\x00_\x00Y\x00:\x00 \x001\x002\x005\x00.\x003\x009\x004\x004\x003\x008\x000\x008\x004\x009\x000\x004\x005\x001\x00\n'

In [62]:
props = [''.join(list(map(chr, unpack('>' + 'h'*(len(bs[160:])//2), bs[160:]))))
                for bs in bytestreams]

In [198]:
props

['TM_Quality: 529.5496215820312\nTM_Radius: 0.3\nTM_X: 94.83410954052202\nTM_Y: 125.39443808490451\n',
 '_Quality: 1003.0696411132812\nTM_Radius: 0.3\nTM_X: 94.68939434686278\nTM_Y: 125.42781715918267\n',
 '_Quality: 1087.1341552734375\nTM_Radius: 0.3\nTM_X: 94.87792682283428\nTM_Y: 125.46479753706329\n',
 '_Quality: 990.0999755859375\nTM_Radius: 0.3\nTM_X: 94.75473263782494\nTM_Y: 125.69158879973178\n',
 '_Quality: 809.786376953125\nTM_Radius: 0.3\nTM_X: 94.729591327467\nTM_Y: 125.80808796336997\n',
 '_Quality: 895.5128784179688\nTM_Radius: 0.3\nTM_X: 94.44173924435866\nTM_Y: 125.89441114824943\n',
 'TM_Quality: 766.9214477539062\nTM_Radius: 0.3\nTM_X: 94.45562319002912\nTM_Y: 126.10644703159014\n',
 '_Quality: 731.497802734375\nTM_Radius: 0.3\nTM_X: 94.39651444595575\nTM_Y: 126.10213628564911\n',
 '_Quality: 583.3685913085938\nTM_Radius: 0.3\nTM_X: 94.60191724031117\nTM_Y: 126.00010299580089\n']

In [65]:
hdr[0]

(b'Iout', 227, 2, 962, 727, 967, 732, 0, 0., 0., 0., 0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 64)

In [80]:
hdr2

rec.array([(0, 0, 30, 128, 16, 0, 0, 0, 0, 0),
           (0, 0, 31, 128, 14, 0, 0, 0, 0, 0),
           (0, 0, 32, 128, 14, 0, 0, 0, 0, 0),
           (0, 0, 33, 128, 14, 0, 0, 0, 0, 0),
           (0, 0, 34, 128, 14, 0, 0, 0, 0, 0),
           (0, 0, 35, 128, 14, 0, 0, 0, 0, 0),
           (0, 0, 36, 128, 16, 0, 0, 0, 0, 0),
           (0, 0, 37, 128, 14, 0, 0, 0, 0, 0),
           (0, 0, 38, 128, 14, 0, 0, 0, 0, 0)],
          dtype={'names':['c','z','t','name_offset','name_length','label_color','font_size','opacity','image_size','float_stroke_width'], 'formats':['>i4','>i4','>i4','>i4','>i4','>i4','>i2','i1','>i4','>i4'], 'offsets':[4,8,12,16,20,24,28,31,32,36], 'itemsize':40})

In [67]:
512 | 256

768

In [81]:
hdr[0]

(b'Iout', 227, 2, 962, 727, 967, 732, 0, 0., 0., 0., 0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 64)

In [102]:
64+48+16

128

In [85]:
unpack('>' + 'i'*(len(bytestreams[0][104:])//4), bytestreams[0][104:])

(160,
 94,
 0,
 0,
 0,
 0,
 3145776,
 3342384,
 2949168,
 3145779,
 3145773,
 3145785,
 3538996,
 2293880,
 5505101,
 6226001,
 7667809,
 7077993,
 7602297,
 3801120,
 3473458,
 3735598,
 3473460,
 3735606,
 3276849,
 3473464,
 3276848,
 3342385,
 3276810,
 5505101,
 6226002,
 6357092,
 6881397,
 7536698,
 2097200,
 3014707,
 655444,
 5046367,
 5767226,
 2097209,
 3407918,
 3670067,
 3407921,
 3145785,
 3473460,
 3145781,
 3276850,
 3145778,
 655444,
 5046367,
 5832762,
 2097201,
 3276853,
 3014707,
 3735604,
 3407923,
 3670064,
 3670068,
 3735600,
 3407925,
 3211274)

In [19]:
with roi_read.IJZipReader(regexp=r'.*[0-9](#x)?\.roi$', sep=None) as multipoint:
    multipoint.read(r'C:\Users\v\Anaconda3\envs\tensorflow\projects\endocytosis\endocytosis\test\data\test_multipoint_roi.zip')

TypeError: object() takes no parameters

In [35]:
bytestreams = multipoint.bytestreams

In [59]:
hdr = np.recarray(2, dtype=HEADER_DTYPE, buf=b''.join([m[:64] for m in multipoint.bytestreams]))

In [64]:
coords = [bs[64+n*2*2:64+n*2*6] for bs, n in zip(bytestreams, hdr['n_coordinates'])]

In [67]:
y = [unpack('>'+ 'f'*(len(c)//8), c[:len(c)//2]) for c in coords]

In [68]:
x = [unpack('>'+ 'f'*(len(c)//8), c[len(c)//2:]) for c in coords]

In [71]:
for i, j in zip(x, y): 
    print('x: {}'.format(i))
    print('y: {}'.format(j))
    print('')

x: (398.6666564941406, 249.3333282470703, 285.3333435058594, 360.0, 402.66668701171875, 668.0, 410.66668701171875, 298.6666564941406, 194.6666717529297, 360.0, 750.6666870117188, 816.0, 422.66668701171875, 190.6666717529297, 301.3333435058594, 442.66668701171875, 597.3333129882812, 709.3333740234375)
y: (408.0, 272.0, 273.3333435058594, 400.0, 502.66668701171875, 216.0, 633.3333740234375, 206.6666717529297, 497.33331298828125, 557.3333129882812, 561.3333129882812, 574.6666870117188, 605.3333129882812, 214.6666717529297, 397.33331298828125, 601.3333129882812, 281.3333435058594, 444.0)

x: (398.6666564941406, 249.3333282470703, 285.3333435058594, 360.0, 402.66668701171875, 668.0, 410.66668701171875, 298.6666564941406, 194.6666717529297, 360.0, 750.6666870117188, 816.0, 422.66668701171875, 190.6666717529297, 301.3333435058594, 442.66668701171875, 597.3333129882812, 709.3333740234375, 386.66668701171875, 245.3333282470703, 592.0, 714.6666870117188, 446.66668701171875, 353.33331298828125, 6

In [73]:
hdr2 = np.recarray(2, dtype=HEADER2_DTYPE, buf=b''.join([bs[off:off+52] for bs, off in zip(bytestreams, hdr['hdr2_offset'])]))

In [74]:
hdr2

rec.array([(1, 5,  3, 344, 14, 0, 0, 0, 0, 0, 0, 0, 372),
           (1, 6, 61, 464, 14, 0, 0, 0, 0, 0, 0, 0, 492)],
          dtype={'names':['c','z','t','name_offset','name_length','overlay_label_color','overlay_font_size','image_opacity','image_size','float_stroke_width','roi_props_offset','roi_props_length','counters_offset'], 'formats':['>i4','>i4','>i4','>i4','>i4','>i4','>i2','i1','>i4','>i4','>i4','>i4','>i4'], 'offsets':[4,8,12,16,20,24,28,31,32,36,40,44,48], 'itemsize':52})

In [79]:
off = 372
n = 18
bs = bytestreams[0]

data = unpack('>'+'hbb'*(n-1) + 'hb', bs[off+1:off+n*4])
positions = data[::3]
counters = data[1::3]


# unpack('>' + 'i'*18, bytestreams[0][372:372+18*4])

In [80]:
positions

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17)

In [81]:
counters

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [54]:
np.recarray(2, dtype=HEADER2_DTYPE, buf=b''.join([bs[off:off+52] for off, bs in zip(hdr['hdr2_offset'], multipoint.bytestreams)]))

rec.array([(4391105, 19398665, 27983872, 19071327, 23265648, 26148872, 191, -117, 4915437, 13631547,  6226089, 13894109, 14418028),
           (4391105, 19398665, 27983872, 19071327, 23265648, 26148872, 191, -117, 4915437,  8126761, 33292453, 11272579, 22282559)],
          dtype={'names':['c','z','t','name_offset','name_length','overlay_label_color','overlay_font_size','image_opacity','image_size','float_stroke_width','roi_props_offset','roi_props_length','counters_offset'], 'formats':['>i4','>i4','>i4','>i4','>i4','>i4','>i2','i1','>i4','>i4','>i4','>i4','>i4'], 'offsets':[4,8,12,16,20,24,28,31,32,36,40,44,48], 'itemsize':52})

In [121]:
np.recarray.view?

In [125]:
COLOR_DTYPE = np.dtype(dict(
    names=['alpha', 'red', 'green', 'blue'],
    offsets=[0, 1, 2, 3],
    formats=['b', 'b', 'b', 'b']))

In [126]:
np.array([-256]).view(COLOR_DTYPE)

array([(0, -1, -1, -1)],
      dtype=[('alpha', 'i1'), ('red', 'i1'), ('green', 'i1'), ('blue', 'i1')])

In [146]:
np.array([1, 0, 0, 0]).view(COLOR_DTYPE)['alpha'].view(bool)

array([ True, False, False, False])

In [140]:
arr = np.array([True, False, False, False])

In [145]:
HEADER2_DTYPE.byteorder

'|'

In [147]:
np.dtype({'names': ['col1', 'col2'], 'formats': ['i4','f4']})

dtype([('col1', '<i4'), ('col2', '<f4')])

In [150]:
np.dtype({'names': ['col1', ['new', ['a', 'b', 'c']], 'col2'], 'formats': ['i4', ['i4', 'i4', 'i4'], 'f4']})

TypeError: data type not understood

In [157]:
import datetime

In [176]:
d = datetime.date.today()

In [190]:
str(d.toordinal())

'736923'

In [188]:
unpack('>i', pack('>i', d.toordinal()*4))

(2947692,)

In [189]:
datetime.date.fromordinal(d.toordinal()*4)

datetime.date(8071, 7, 4)

In [195]:
'YYYYMMDD: {}'.format(datetime.date.fromordinal(d.toordinal()))

'YYYYMMDD: 2018-08-17'

In [197]:
d.year

2018

In [214]:
author = b'Author of Package for encoding ImageJ ROI: Vladimir Shteyn\n'

In [234]:
bytestreams[0]

b'Iout\x00\xe3\x02\x00\x03\xc2\x02\xd7\x03\xc7\x02\xdc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x1e\x00\x00\x00\x80\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0\x00\x00\x00^\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000\x000\x003\x000\x00-\x000\x000\x003\x000\x00-\x000\x009\x006\x004\x00#\x00x\x00T\x00M\x00_\x00Q\x00u\x00a\x00l\x00i\x00t\x00y\x00:\x00 \x005\x002\x009\x00.\x005\x004\x009\x006\x002\x001\x005\x008\x002\x000\x003\x001\x002\x00\n\x00T\x00M\x00_\x00R\x00a\x00d\x00i\x00u\x00s\x00:\x00 \x000\x00.\x003\x00\n\x00T\x00M\x00_\x00X\x00:\x00 \x009\x004\x00.\x008\x003\x004\x001\x000\x009\x005\x004\x000\x005\x002\x002\x000\x002\x00\n\x00T\x00M\x00_\x00Y\x00:\x00 \x001\x002\x005\x00.\x003\x009\x004\x004\x003\

In [87]:
np.ceil(np.array((2.0,)))

array([2.])

In [94]:
256+128 & 128

128

In [95]:
bytestreams[0] + bytestreams[1]

b"Iout\x00\xe3\n\x00\x00\xbe\x00\xce\x031\x02z\x00\x12\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x18\x00\xc9\x00A\x00C\x00\xc1\x01(\x00\t\x01\xab\x00\x00\x01#\x01_\x01c\x01p\x01\x8f\x00\x08\x00\xbf\x01\x8b\x00K\x00\xed\x00\xd0\x00;\x00_\x00\xa9\x00\xd4\x01\xdd\x00\xdc\x00l\x00\x04\x00\xa9\x020\x02q\x00\xe8\x00\x00\x00o\x00\xfc\x01\x97\x02\x07C\xcc\x00\x00C\x88\x00\x00C\x88\xaa\xabC\xc8\x00\x00C\xfbUVCX\x00\x00D\x1eUVCN\xaa\xabC\xf8\xaa\xaaD\x0bUUD\x0cUUD\x0f\xaa\xabD\x17UUCV\xaa\xabC\xc6\xaa\xaaD\x16UUC\x8c\xaa\xabC\xde\x00\x00C\xc7UUCyUUC\x8e\xaa\xabC\xb4\x00\x00C\xc9UVD'\x00\x00C\xcdUVC\x95UUCB\xaa\xabC\xb4\x00\x00D;\xaa\xabDL\x00\x00C\xd3UVC>\xaa\xabC\x96\xaa\xabC\xddUVD\x15UUD1UV\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x05\x00\x00\x00\x03\x00\x00\x01X\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [99]:
hdr[['version', 'type']]

rec.array([(227, 10), (227, 10)],
          dtype=[('version', '>i2'), ('type', '<i2')])

In [103]:
multipoint.cleanup()

In [104]:
multipoint._file.open

<bound method ZipFile.open of <zipfile.ZipFile [closed]>>